In [ ]:
import os
import geopandas as gp
import pandas as pd
import maup
import numpy as np
maup.progress.enabled = False
import warnings
import time 
import zipfile
warnings.filterwarnings("ignore")

In [ ]:
wd = os.getcwd()
block_fold = os.path.join(wd,'extracted_blocks')
prec_fold = os.path.join(wd,'precs_2020')

In [ ]:
block_dict = {}
for folder in os.listdir(block_fold):
    fp = os.path.join(block_fold,folder)
    for file in os.listdir(fp):
        if file.endswith('.shp'):
            sa = file.split('_')[0]
            #print('on: ', sa)
            gdf = gp.read_file(os.path.join(fp,file))
            #print(sa, ' read!')
            block_dict.update({sa:gdf})

In [ ]:
prec_dict = {}
for folder in os.listdir(prec_fold):
    fp = os.path.join(prec_fold,folder)
    for file in os.listdir(fp):
        if file.endswith('.shp'):
            sa = file.split('_')[0]
            #print('on: ',sa)
            gdf = gp.read_file(os.path.join(fp,file))
            #print(sa, ' read!')
            prec_dict.update({sa:gdf})

In [ ]:
def get_block_match(prec_sa,block_dict=block_dict):
    block_gdf = block_dict.get(prec_sa)
    return block_gdf

In [ ]:
def check_valid_rows(block_gdf, precinct_gdf):
    prec_geom = precinct_gdf.geometry
    valid_rows = precinct_gdf[~(prec_geom.isna() | prec_geom.is_empty)]
    valid_rows = valid_rows[valid_rows.is_valid]
    if len(valid_rows)!=len(precinct_gdf):
        #print('There are rows that are NOT valid in the precinct file!')
        not_valid_rows = precinct_gdf[(prec_geom.isna() | prec_geom.is_empty)]
        not_valid_rows = not_valid_rows[~not_valid_rows.is_valid]
        #display(not_valid_rows)
        
    block_geom = block_gdf.geometry
    block_valid_rows = block_gdf[~(block_geom.isna() | block_geom.is_empty)]
    block_valid_rows = block_valid_rows[block_valid_rows.is_valid]
    if len(block_valid_rows)!=len(block_gdf):
        #print('There are rows that are NOT valid in the block file!')
        not_valid_block = block_gdf[(block_geom.isna() | block_geom.is_empty)]
        not_valid_block = not_valid_block[~not_valid_block.is_valid]
        #display(not_valid_block)

    #print('Valid rows check complete!')

In [ ]:
def fix_buffer(gdf):
    """
    return (GeoDataFrame) with the 'bufer(0) trick' applied
    :gdf: (GeoDataFrame) object
    Can be useful when trying to mitigate 'self-intersection' issues
    """
    #buffered = gdf.buffer(0)
    #gdf.drop(columns=["geometry"])
    # gdf['geometry'] = gdf.apply(lambda x: x.geometry.buffer(0), axis=1)
    #gdf["geometry"] = buffered
    
    gdf['geometry'] = gdf['geometry'].buffer(0)
    return gdf

In [ ]:
def column_total_check(election_columns, block_gdf, precinct_gdf):
    for val in election_columns:
        #print(block_gdf.value_counts)
        vote_dif = block_gdf[val].sum()-precinct_gdf[val].sum()
        if (abs(vote_dif) <=1e-1):
            print(val+": EQUAL", ' - total: ', 'block:', str(block_gdf[val].sum()), 'prec:', str(precinct_gdf[val].sum()), 'diff:', block_gdf[val].sum()-precinct_gdf[val].sum())
        else:
            print(val+": DIFFERENCE OF " + str(vote_dif)+ " VOTES", ' - block total: ', str(block_gdf[val].sum()), ', precinct total: ', str(precinct_gdf[val].sum()))

In [ ]:
def get_unique_col(precincts,election_cols):
    elec_copy = election_cols
    unique_id = False
    election_cols.append('geometry')
    for i in list(precincts.columns):
        if len(precincts[i].unique())==len(precincts):
            if i not in elec_copy:
                unique_id = i
                #print('Unique ID is: ' ,unique_id)
                break
    if unique_id == False:
        dict_of_lens ={}
        for i in precincts.columns:
            if i not in elec_copy:
                dict_of_lens.update({i:len(precincts[i].unique())})
        max1_len=0
        max2_len=0
        max1=False
        max2=False
        for k,v in dict_of_lens.items():
            if v>max1_len:
                max2_len = max1_len
                max2 = max1
                max1_len = v
                max1 = k

            elif v>max2_len:
                max2_len = v
                max2 = k
            else:
                continue
        unique_id = str(max1)+'_'+str(max2)
        #print('Unique ID is: ', unique_id)
        precincts[unique_id] = precincts.apply(lambda x: str(x[max1])+ ' - '+str(x[max2]),axis=1)
    if len(precincts[unique_id].unique())!=len(precincts):
        precincts['counter'] = range(len(precincts))
        precincts[unique_id] = precincts.apply(lambda x: str(x[unique_id]) + ' - ' + str(x['counter']),axis=1)
    return unique_id

In [ ]:
def get_election_cols(precincts):
    election_columns = []
    for i in list(precincts.columns):
        if i.startswith('G20'):
            election_columns.append(i)
    #print('Election columns: ', election_columns)
    return election_columns

In [ ]:
def check_and_remove_null_blocks(blocks,precincts,unique_id):
    if len(blocks[blocks['assignment'].isna()])!=0:
        #print('There are blocks with NO assignment!')
        na_blocks = blocks[blocks['assignment'].isna()]
        na_blocks_not_zero = na_blocks[na_blocks['VAP_MOD']>0]
        display(na_blocks_not_zero)
        #print(list(na_blocks_not_zero['GEOID20']))
        blocks = blocks[~blocks['assignment'].isna()]
    blocks['PRECINCT_ID'] = blocks['assignment'].map(lambda x: str(precincts.loc[x][unique_id]))
    return blocks

In [ ]:
def check_missing_precs(precincts,blocks,unique_id):
    not_in_blocks = list(set(precincts[unique_id]) - set(blocks['PRECINCT_ID']))
    if not_in_blocks!=list(set()):
        #print('Precincts that did NOT make it to any blocks: ', not_in_blocks)
        print("")
    return not_in_blocks

In [ ]:
def agg_vap_to_precs(blocks,precincts,assignment):
    precincts['VAP_MOD'] = blocks['VAP_MOD'].groupby(assignment).sum()
    return precincts

In [ ]:
def get_precs_with_no_vap(precincts):
    precincts_with_no_pop = precincts.index[precincts['VAP_MOD'].isin([0,0.0])].tolist()
    #print('Precincts with no popultion: ', precincts_with_no_pop)
    return precincts_with_no_pop

In [ ]:
def make_block_vap_1(precincts_with_no_pop,blocks,precincts):
    blocks_to_mod = blocks[blocks['assignment'].isin(precincts_with_no_pop)]['GEOID20'].to_list()
    blocks['VAP_MOD'] = blocks.apply(lambda x: 1 if x['GEOID20'] in blocks_to_mod else x['VAP_MOD'],axis=1)
    #print('Block modification complete!')
    return blocks, blocks_to_mod

In [ ]:
def clean_files(precincts,blocks):
    check_valid_rows(blocks,precincts)
    fix_buffer(precincts)
    fix_buffer(blocks)
    precincts = precincts.to_crs(blocks.crs) 
    return precincts,blocks

In [ ]:
def allocate_votes(precincts,blocks,unique_id,election_columns,assignment):
    not_in_blocks = check_missing_precs(precincts,blocks,unique_id)
    precincts = agg_vap_to_precs(blocks, precincts,assignment)
    precincts_with_no_pop = get_precs_with_no_vap(precincts)
    vap_output = make_block_vap_1(precincts_with_no_pop,blocks,precincts)
    blocks = vap_output[0]
    blocks_to_mod = vap_output[1]
    precincts = agg_vap_to_precs(blocks,precincts,assignment)
    weights = blocks.VAP_MOD/assignment.map(precincts.VAP_MOD)
    print('Weights complete!')
    if 'geometry' in election_columns:
        election_columns.remove('geometry')
    blocks[election_columns] = maup.prorate(assignment, precincts[election_columns], weights)
    print('Proration complete!')
    return precincts,blocks,blocks_to_mod,precincts_with_no_pop

In [ ]:
def maup_assign(blocks,precincts):
    try:
        assignment = maup.assign(blocks, precincts)
        blocks['assignment'] = assignment
        print('MAUP complete!')
        return blocks, assignment
    except ValueError as v:
        print(blocks.is_valid)
        print(precincts.is_valid)
        print(v)
        print('Could not run MAUP!')
    except IndexError as i:
        print(blocks.is_valid)
        print(precincts.is_valid)
        print(v)
        print('Could not run MAUP!')
    except TypeError as t:
        print(blocks.is_valid)
        print(precincts.is_valid)
        print(t)
        print('Could not run MAUP!')
    except:
        try:
            precincts['geometry'] = precincts['geometry'].buffer(0)
            blocks['geometry'] = blocks['geometry'].buffer(0)
            assignment = maup.assign(blocks, precincts)
            blocks['assignment'] = assignment
            print('MAUP complete!')
            return blocks,assignment
        except:
            print('Still could not run MAUP!')
        


In [ ]:
def check_sums_by_precinct(precincts,blocks,unique_id,election_columns):
    election_col = election_columns.copy()
    election_col.append('VAP_MOD')
    #print('STARTING ADDITIONAL SUM BY PRECINCT CHECK')
    blocks_grouped = blocks.groupby('PRECINCT_ID').sum()
    blocks_grouped.reset_index(inplace=True,drop=False)
    for i in blocks_grouped['PRECINCT_ID']:
        sub = blocks_grouped[blocks_grouped['PRECINCT_ID']==i]
        precinct_sub = precincts[precincts[unique_id]==i]
        for col in election_col:
            if round(sub[col].sum()) != round(precinct_sub[col].sum()):
                continue
                print("Blocks sum to: ", sub[col].sum(), ' for ', col, ' in precinct: ', i)
                print("Precincts sum to: ",precinct_sub[col].sum(), ' for ', col, ' in precinct: ', i)
                print('')   
    #print('SUM BY PRECINCT CHECK COMPLETE!')

In [ ]:
def allocate_missing_precincts(precincts,blocks,unique_id,blocks_to_mod,election_columns,precincts_with_no_pop):
    not_in_blocks = check_missing_precs(precincts,blocks,unique_id)
    if len(not_in_blocks) !=0:
        #print('Missing precincts: ', not_in_blocks)
        missing_precs = precincts[precincts[unique_id].isin(not_in_blocks)]
        missing_precs['total_votes'] = missing_precs[election_columns].sum(axis=1)
        missing_precs = missing_precs[~missing_precs['total_votes'].isin([0,0.0])]
        if len(missing_precs)!=0:
            b_clipped = gp.clip(blocks,missing_precs)
            b_clipped = b_clipped[b_clipped.geom_type.isin(['Polygon','MultiPolygon'])]
            allocation_dict = {}
            for i in missing_precs[unique_id]:
                sub_prec = missing_precs[missing_precs[unique_id]==i]
                b_sub = gp.clip(b_clipped,sub_prec)
                b_sub['area'] = b_sub.area
                b_sub= b_sub.nlargest(1,'area')
                geoid = str(list(b_sub['GEOID20'])[0])
                allocation_dict.update({i: geoid})
            #print('ALLOCATION DICTIONARY: ', allocation_dict)

            for i in missing_precs[unique_id]:
                #print('Starting: ', i)
                sub_prec = missing_precs[missing_precs[unique_id] == i]
                #display(sub_prec)
                block_match = allocation_dict.get(i)
                #print('Block match: ', block_match)
                blocks.loc[blocks['GEOID20'] == block_match, 'PRECINCT_ID'] = ' & '.join([str(list(blocks[blocks['GEOID20'] == block_match]['PRECINCT_ID'])[0]), i])
                #display(blocks[blocks['GEOID20']==block_match])
                for col in election_columns:
                    blocks_col_cur_sum = blocks[blocks['GEOID20'] == block_match][col].sum()
                    #print('Current sum for ', col, ': ', blocks_col_cur_sum)
                    blocks.loc[blocks['GEOID20'] == block_match, col] = sub_prec[col].sum() + blocks_col_cur_sum
                    #print('New sum for ', col, ': ',  sub_prec[col].sum() + blocks_col_cur_sum)
    blocks['VAP_MOD'] = blocks.apply(lambda x: 0 if x['GEOID20'] in blocks_to_mod else x['VAP_MOD'],axis=1)
    try:
        precincts['VAP_MOD'] = precincts.apply(lambda x: 0 if x.index in precincts_with_no_pop else x['VAP_MOD'],axis=1)
    except:
        precincts['index'] = precincts.index
        precincts['VAP_MOD'] = precincts.apply(lambda x: 0 if x['index'] in precincts_with_no_pop else x['VAP_MOD'],axis=1)
        precincts.drop(columns = 'index',inplace=True)
    check_sums_by_precinct(precincts,blocks,unique_id,election_columns)
    print('Missing precincts allocation complete!')
    return blocks,precincts

In [ ]:
def fill_and_sort(blocks,original_blocks):
    if len(blocks)!=len(original_blocks):
        missing_blocks = original_blocks[~original_blocks['GEOID20'].isin(list(blocks['GEOID20']))].copy(deep=True)
        blocks = gp.GeoDataFrame(pd.concat([blocks,missing_blocks]),crs=blocks.crs)
        blocks.fillna({'PRECINCT_ID': 'N/A'}, inplace=True)
    blocks.fillna(0,inplace=True)
    blocks.drop(columns='assignment',inplace=True)
    blocks_list = list(blocks.columns)
    blocks_list.remove('GEOID20')
    blocks_list.remove('PRECINCT_ID')
    blocks_list.remove('VAP_MOD')
    blocks_list.remove('geometry')
    blocks['STATEFP'] = blocks['GEOID20'].apply(lambda x: str(x[:2]))
    blocks['COUNTYFP'] = blocks['GEOID20'].apply(lambda x: str(x[2:5]))
    blocks.rename(columns = {'PRECINCT_ID':'PRECINCTID'},inplace=True)
    col_order = ['GEOID20','STATEFP','COUNTYFP','PRECINCTID','VAP_MOD'] + blocks_list + ['geometry']
    blocks = blocks[col_order]
    blocks.sort_values(by='GEOID20',inplace=True)
    return blocks

In [ ]:
def check_vap(blocks,orig_blocks):
    column_total_check(['VAP_MOD'],blocks,orig_blocks)

In [ ]:
def check_totals_original (block,prec):
    display(block.head())
    display(prec.head())
    cols_to_check = get_election_cols(prec)
    column_total_check(cols_to_check,block,prec)

In [ ]:
def append_assignment(blocks,precincts,assignment,sa):
    if sa == 'wa':
        bindex = blocks.index[blocks['GEOID20'] == '530299701002015'].tolist()[0]
        print(assignment.iloc[bindex])
        print('Block index to modify in Washington: ', bindex)
        pindex = float(precincts.index[precincts['PRECNAME'] == 'N WHIDBEY 01'].tolist()[0])
        print('Precinct index to modify in Washington: ', pindex)
        assignment.update(pd.Series([pindex], index=[bindex]))
        print('Assignment updated!')
        print(assignment.iloc[bindex])
        blocks['assignment'] = assignment
        return blocks,assignment
    elif sa == 'or':
        bindex = blocks.index[blocks['GEOID20'] == '410579601021095'].tolist()[0]
        print('Block index to modify in Oregon: ', bindex)
        pindex = float(precincts.index[precincts['NAME'] == 'FOLEY'].tolist()[0])
        print(assignment.iloc[bindex])
        print('Precinct index to modify in Oregon: ', pindex)
        assignment.update(pd.Series([pindex], index=[bindex]))
        print('Assignment updated!')
        print(assignment.iloc[bindex])
        blocks['assignment'] = assignment
        return blocks,assignment
    else:
        return blocks,assignment


In [ ]:
def zip_files(src, dst):
    zf = zipfile.ZipFile("%s.zip" % (dst), "w", zipfile.ZIP_DEFLATED)
    abs_src = os.path.abspath(src)
    for dirname, subdirs, files in os.walk(src):
        for filename in files:
            absname = os.path.abspath(os.path.join(dirname, filename))
            arcname = absname[len(abs_src) + 1:]
            #print ('zipping %s as %s' % (os.path.join(dirname, filename),arcname))
            zf.write(absname, arcname)
    zf.close()

In [ ]:
def zip_directory(wd):
    for i in os.listdir(wd):
        src = os.path.join(wd,i)
        dest_fold = os.path.join(os.getcwd(),'zipped')
        if not os.path.exists(dest_fold):
            os.mkdir(dest_fold)
        dest = os.path.join(dest_fold,i)
        zip_files(src,dest)

In [ ]:
def extract_data(maup_dict):
    extract_folder = os.path.join(wd,'extracted_disag')
    if not os.path.exists(extract_folder):
        os.mkdir(extract_folder)
    for k,v in maup_dict.items():
        start = time.time()
        print('starting: ', k)
        name = k+'_2020_gen_2020_blocks'
        folder_name = os.path.join(extract_folder,name)
        if not os.path.exists(folder_name):
            os.mkdir(folder_name)
        path = os.path.join(folder_name,name+'.shp')
        for col in v.columns:
            if col.startswith('G20'):
                v[col] = v[col].apply(lambda x: round(x,2))
        display(v.head(1))
        v.to_file(path)
        stop = time.time()
        calc_time = round((stop-start)/60,2)
        print(k,' is extracted in ', str(calc_time), ' minutes!')
        zip_directory(extract_folder)

In [ ]:
def run_maup(precinct_name,precincts):
    print('starting: ',precinct_name)
    blocks = get_block_match(precinct_name)
    original_blocks = blocks.copy(deep=True)
    cleaned_outputs = clean_files(precincts,blocks)
    precincts = cleaned_outputs[0]
    blocks = cleaned_outputs[1]
    election_columns = get_election_cols(precincts)
    unique_id = get_unique_col(precincts,election_columns)
    assign_out = maup_assign(blocks,precincts)
    blocks = assign_out[0]
    assignment = assign_out[1]
    
    reassign = append_assignment(blocks,precincts,assignment,precinct_name)
    blocks = reassign[0]
    assignment = reassign[1]
    
    blocks = check_and_remove_null_blocks(blocks,precincts,unique_id)
    allocated_outputs = allocate_votes(precincts,blocks,unique_id,election_columns,assignment)
    precincts = allocated_outputs[0]
    blocks = allocated_outputs[1]
    blocks_to_mod = allocated_outputs[2]
    precincts_with_no_votes = allocated_outputs[3]
    check_sums_by_precinct(precincts,blocks,unique_id,election_columns)
    reallocated = allocate_missing_precincts(precincts,blocks,unique_id,blocks_to_mod,election_columns,precincts_with_no_votes)
    blocks = reallocated[0]
    precicts = reallocated[1]
    blocks = fill_and_sort(blocks,original_blocks)
    column_total_check(election_columns,blocks,precincts)
    check_vap(blocks,original_blocks)
    print('DISAG IS COMPLETE FOR: ', precinct_name.upper())
    return blocks

In [ ]:
def iterate_maup(prec_dict=prec_dict):
    maup_gdfs = {}
    for k,v in prec_dict.items():
        blocks = run_maup(k,v)
        maup_gdfs.update({k:blocks})
        #display(v.head(1))
        check_totals_original(blocks,v)
        print('*****************************************************************')
    return maup_gdfs

In [ ]:
maup_dict = iterate_maup(prec_dict)

In [ ]:
extract_data(maup_dict)